In [1]:
# AOC - Day14 - Part 2
# https://adventofcode.com/2019/day/14#part2

In [2]:
import math
import time

In [3]:
class Chemical:
    # need to do something special when this chemical is made
    # this is the ore, which gets mined, not made
    global o_str
    global ch
    def __init__(self, name, qty, rqty, reactants):
        self.name = name
        self.qty = qty
        self.rqty = rqty
        self.reactants = reactants
        return
    def make(self, req_qty):
        global ore_qty
        # if ore is being requested
        # print("{}.make({})...".format(self.name, req_qty))
        if(self.name == o_str):
            # add this amount to accumulator
            ore_qty = ore_qty + req_qty
            # and return what is requested (instantly mined!)
            # print("{} ORE mined! total so far: {}".format(req_qty, ore_qty))
            return(req_qty)
        # so ore not being requested.
        # see what is in inventory, and only kick off the reaction for what inventory can't provide
        need_amt = req_qty - self.qty
        if(need_amt <= 0):
            # have enough on hand.  reduce the inventory and return the requested amount.
            # print("...enough on hand: {}".format(self.qty))
            self.qty -= req_qty
            return(req_qty)
        else:
            # determine factor required given need_amt and unit_quantity in the make reaction
            factor = calc_factor(need_amt, self.rqty)
            # now 'do the reaction' - iterate through the reactants
            for r in self.reactants:
                # get the ch array element for this reactant
                c = find_chem_by_name(ch, symnum(r)[0])
                req_amt = symnum(r)[1] * factor
                # print("{} reactant: {} unit: {}, factor: {}, req_amt: {}, qty on-hand before make: {}".format(self.name, symnum(r)[0], symnum(r)[1], factor, req_amt, c.qty))
                made_amt = c.make(req_amt)
                # update on-hand leftover (req_amt is consumed in the calling reaction)
                c.qty = c.qty + made_amt - req_amt
                # print("{} reactant: {} made_amt: {}, qty on-hand after make: {}".format(self.name, symnum(r)[0], made_amt, c.qty))
        return(factor*self.rqty)

In [4]:
class Reactant:
    def _init__(self, chem, qty):
        self.chem = chem
        self.qty = qty
        return

class Reaction:
    def __init__(self, reaction_tup):
        self.reactant_list = reaction_tup[0]
        self.product = reaction_tup[1]
        return

In [5]:
def parse_reaction(r_txt):
    # r_txt comes in like: "44 XJWVT, 5 KHKGT, 1 QDVJ, 29 NZVS, 9 GPVTF, 48 HKGWZ => 1 FUEL"
    # parse this out and return a reactant list (r_list) and product (p)
    # each looks like  {'CHEM_SYMBOL': NUMBER}
    r_list = []
    p = {}
    left, right = r_txt.split(" => ")
    p_num, p_chem = right.split()
    p[p_chem] = int(p_num)
    for l in left.split(", "):
        reactant = {}
        reactant[l.split()[1]] = int(l.split()[0])
        r_list.append(reactant)
    return(r_list, p)

In [6]:
def calc_factor(req, makeunit):
    return(int(math.ceil(req / makeunit)))

In [7]:
# this function returns the first key, value of a dictionary as a tuple
def symnum(r):
    sym = list(r.keys())[0]
    num = list(r.values())[0]
    return((sym, num))

In [8]:
def find_chem_by_name(ch, name):
    c = [chem for chem in ch if chem.name == name]
    return(c[0])

In [9]:
# array of chemicals
ch = []
ore_qty = 0
# these are the strings used for the chemical symbols for Fuel and Ore
f_str = "FUEL"
o_str = "ORE"

In [10]:
# read the reactions list from the input file
ifn = "day14-input.txt"
# parse each line
# each line defines a chemical and contains the reactancts
with open(ifn) as f:
    for line in f:
        r_list, p = parse_reaction(line)
        # create a chemical for the product and add it to the chemical array
        # initial quantity is 0, of course
        ch.append(Chemical(symnum(p)[0], 0, symnum(p)[1], r_list))
# add ore as a reaction (ore => ore)
o_reaction_line = "1 {} => 1 {}".format(o_str, o_str)
r_list, p = parse_reaction(o_reaction_line)
ch.append(Chemical(symnum(p)[0], 0, symnum(p)[1], r_list))

In [11]:
# this will return the chemical element that is the fuel
x = find_chem_by_name(ch, f_str)

In [12]:
one_trillion = 1000000000000
# ore_qty_max = 1000000000000
ore_qty_max = one_trillion
step = 1000
fuel_cnt = 0
leftover_inv = 1
# make fuel until there are no leftovers of reactants after a fuel is made (or you hit the max)
# calling this a 'clean unit'
starttime = time.perf_counter()
laptime = time.perf_counter()
while(leftover_inv != 0 and ore_qty < ore_qty_max):
    fuel_cnt += x.make(1)
    if(not (fuel_cnt % step)):
        ore_rate = ore_qty / (time.perf_counter() - starttime)
        secsleft = (ore_qty_max - ore_qty) / ore_rate
        print("fuel_cnt: {}, ore_qty: {}, laptime: {}, ore_rate (/s): {}, secs left: {}".format(fuel_cnt, ore_qty, time.perf_counter() - laptime, ore_rate, secsleft))
        laptime = time.perf_counter()
    # now get a sum of leftovers
    leftover_inv = 0
    for c in ch:
        leftover_inv += c.qty
    if(leftover_inv == 0):
        print("no leftovers.  fuel_cnt: {}, ore_qty: {}".format(fuel_cnt, ore_qty))

if(leftover_inv != 0):
    print("no clean unit can be determined.  but we did use up all the ore to make: {} fuels.".format(fuel_cnt-1))
else:
    clean_ore_qty = ore_qty
    # this is the # of clean_ore_units in a trillion
    clean_units_in_a_trillion = int(one_trillion / ore_qty)
    # and this is the # of fuels in a 'clean unit'
    clean_fuel_qty = fuel_cnt
    # how more ore is left after dividing by clean_ore_qty
    remaining_ore = one_trillion % clean_ore_qty
    # now use that remaining amount of ore and see how much fuel can be made
    ore_qty = 0
    fuel_cnt = 0
    while(ore_qty < remaining_ore):
        fuel_cnt += x.make(1)
    # calculate the fuel from even # of clean units, then from the leftover
    fuel_from_a_trillion_ore = clean_units_in_a_trillion * clean_fuel_qty + fuel_cnt - 1

    print("the amount of fuel that can be made using {} units of ore is: {}".format(one_trillion, fuel_from_a_trillion_ore))


fuel_cnt: 1000, ore_qty: 83074584, laptime: 4.925603044976015, ore_rate (/s): 16865831.783079203, secs left: 59286.54680519081
fuel_cnt: 2000, ore_qty: 166134067, laptime: 4.337725924007827, ore_rate (/s): 17934245.649373516, secs left: 55749.981654116935
fuel_cnt: 3000, ore_qty: 249192203, laptime: 4.3460021810024045, ore_rate (/s): 18309937.283828974, secs left: 54601.541900417265
fuel_cnt: 4000, ore_qty: 332257096, laptime: 4.293464808986755, ore_rate (/s): 18558470.75463098, secs left: 53865.846821163774
fuel_cnt: 5000, ore_qty: 415314734, laptime: 4.132635087997187, ore_rate (/s): 18847054.757547084, secs left: 53036.652045897405
fuel_cnt: 6000, ore_qty: 498376250, laptime: 4.133265026990557, ore_rate (/s): 19044160.507384002, secs left: 52483.36482789371
fuel_cnt: 7000, ore_qty: 581437462, laptime: 4.5248012589872815, ore_rate (/s): 18942710.813899435, secs left: 52760.060181284345
fuel_cnt: 8000, ore_qty: 664499559, laptime: 4.147229749010876, ore_rate (/s): 19071840.667080358, 

KeyboardInterrupt: 